In [1]:
from torchtext import data, datasets
import torch
from sklearn.model_selection import train_test_split
import random
import numpy as np
import string
from collections import defaultdict
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import nltk
from nltk.corpus import stopwords
import seaborn as sns

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import numpy as np
from collections import defaultdict, Counter
import string

In [3]:
from sklearn.model_selection import KFold

In [4]:
# For tokenization
TEXT = data.Field(tokenize='spacy',
    tokenizer_language='en_core_web_sm',
    batch_first=True,
    fix_length=50,
    lower=True,
    unk_token='<unk>',
    pad_token='<pad>'
    )

# For multi-class classification labels
LABEL = data.LabelField()

# Load the TREC dataset
train_data, test_data = datasets.TREC.splits(TEXT, LABEL, fine_grained=False)

downloading train_5500.label


.data\trec\train_5500.label: 100%|██████████| 336k/336k [00:01<00:00, 276kB/s]  


downloading TREC_10.label


.data\trec\TREC_10.label: 100%|██████████| 23.4k/23.4k [00:00<00:00, 107kB/s] 


In [5]:
label_set = set()
for i in train_data:
    label_set.add(i.label)

print(label_set)

{'ABBR', 'LOC', 'ENTY', 'NUM', 'DESC', 'HUM'}


In [6]:
TEXT.build_vocab(train_data, vectors="glove.6B.100d")
LABEL.build_vocab(train_data)

.vector_cache\glove.6B.zip: 862MB [06:11, 2.32MB/s]                               
100%|█████████▉| 399999/400000 [00:10<00:00, 37352.49it/s]


In [7]:
embedding_dim = 100
vocab = TEXT.vocab
vectors = vocab.vectors.clone()
glove_vocab = set(vocab.stoi.keys())

# Subword parameters
ngram_min = 3
ngram_max = 6

# Count word frequency in training data
word_counter = Counter()
for example in train_data:
    word_counter.update([w.lower().strip(string.punctuation) for w in example.text])

# Threshold to consider a word “frequent” (adjustable)
freq_threshold = 3

# <unk> vector
unk_vector = torch.zeros(embedding_dim)

def get_subwords(word, n_min=3, n_max=6):
    word = f"<{word.lower()}>"
    subwords = []
    for n in range(n_min, n_max+1):
        subwords += [word[i:i+n] for i in range(len(word)-n+1)]
    return subwords

def get_word_vector(word):
    w_clean = word.lower().strip(string.punctuation)
    
    if w_clean in glove_vocab:
        return vectors[vocab.stoi[w_clean]]
    
    # Subword averaging
    subwords = get_subwords(w_clean, ngram_min, ngram_max)
    subword_vecs = [vectors[vocab.stoi[sg]] for sg in subwords if sg in glove_vocab]
    if subword_vecs:
        return torch.stack(subword_vecs).mean(0)
    
    # Random vector for frequent OOVs
    if word_counter[w_clean] >= freq_threshold:
        return torch.randn(embedding_dim)
    
    # <unk> for rare OOVs
    return unk_vector

# Build embedding matrix
embedding_matrix = {}
for example in train_data:
    for w in example.text:
        if w not in embedding_matrix:
            embedding_matrix[w] = get_word_vector(w)

In [8]:
embedding_dim = 100
vocab = TEXT.vocab

# Create tensor for nn.Embedding
embedding_matrix_tensor = torch.zeros(len(vocab), embedding_dim)
for word, idx in vocab.stoi.items():
    if word in embedding_matrix:
        embedding_matrix_tensor[idx] = embedding_matrix[word]

# Create embedding layer (learnable)
embedding_layer = nn.Embedding.from_pretrained(embedding_matrix_tensor, freeze=False)
print("Embedding layer created with shape:", embedding_layer.weight.shape)


Embedding layer created with shape: torch.Size([8536, 100])


In [ ]:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device:", device)

Using device: cpu


In [26]:
import itertools, copy, pandas as pd


In [13]:
# -----------------------------
# 1. Setup & Config
# -----------------------------
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Assume TEXT, LABEL, train_data, test_data, embedding_layer already exist
train_data, valid_data = train_data.split(split_ratio=0.9, random_state=random.seed(SEED))
print(f"Train: {len(train_data)}, Valid: {len(valid_data)}, Test: {len(test_data)}")

PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
embedding_dim = embedding_layer.weight.size(1)

Train: 4416, Valid: 491, Test: 500


In [ ]:
# -----------------------------
# Simple RNN classifier (tanh)
# -----------------------------
class RNNClassifier(nn.Module):
    def __init__(self, embedding_layer, hidden_dim, num_layers, bidirectional, dropout, num_classes):
        super().__init__()
        self.embedding = embedding_layer
        emb_dim = self.embedding.weight.size(1)
        self.rnn = nn.RNN(
            input_size=emb_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            nonlinearity="tanh",      # simple RNN
            batch_first=True,
            bidirectional=bidirectional,
            dropout=dropout if num_layers > 1 else 0.0
        )
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim * (2 if bidirectional else 1), num_classes)

    def forward(self, text):
        x = self.embedding(text)                     # [B, T, E]
        outputs, hidden = self.rnn(x)                # hidden: [L*D, B, H]
        if self.rnn.bidirectional:
            last = torch.cat((hidden[-2], hidden[-1]), dim=1)  # [B, 2H]
        else:
            last = hidden[-1]                                   # [B, H]
        return self.fc(self.dropout(last))

In [22]:
# -----------------------------
# Train/eval 1 epoch
# -----------------------------
def epoch_run(iterator, model, criterion, optimizer=None):
    model.train(optimizer is not None)
    tot_loss, tot_correct, tot_count = 0.0, 0, 0
    for batch in iterator:
        text, labels = batch.text, batch.label
        logits = model(text)
        loss = criterion(logits, labels)
        if optimizer:
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)  # RNNs can explode
            optimizer.step()
        tot_loss += loss.item() * labels.size(0)
        tot_correct += (logits.argmax(1) == labels).sum().item()
        tot_count += labels.size(0)
    return tot_loss / max(tot_count,1), tot_correct / max(tot_count,1)


In [ ]:
# -----------------------------
# Grid search with early stopping that SAVES BEST WEIGHTS
# -----------------------------
HIDDEN_DIM = 128
NUM_LAYERS = 1
BIDIRECTIONAL = True
DROPOUT = 0.3
NUM_CLASSES = len(LABEL.vocab)
MAX_EPOCHS = 20
PATIENCE = 3

PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

batch_sizes = [32, 64, 128]
learning_rates = [5e-4, 1e-3, 2e-3, 3e-3]

results = []

for bs, lr in itertools.product(batch_sizes, learning_rates):
    print(f"\n🔍 Testing BATCH={bs}, LR={lr}")
    train_iter, valid_iter = data.BucketIterator.splits(
        (train_data, valid_data),
        batch_size=bs,
        sort_key=lambda x: len(x.text),
        sort_within_batch=True,
        device=device
    )

    # fresh embedding weights each trial
    emb_layer = nn.Embedding.from_pretrained(
        embedding_layer.weight.data.clone(), freeze=False, padding_idx=PAD_IDX
    )
    model = RNNClassifier(
        embedding_layer=emb_layer,
        hidden_dim=HIDDEN_DIM,
        num_layers=NUM_LAYERS,
        bidirectional=BIDIRECTIONAL,
        dropout=DROPOUT,
        num_classes=NUM_CLASSES
    ).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    best_val_acc = -1.0
    epochs_no_improve = 0
    best_model_state = None

    for epoch in range(1, MAX_EPOCHS + 1):
        train_loss, train_acc = epoch_run(train_iter, model, criterion, optimizer)
        val_loss, val_acc = epoch_run(valid_iter, model, criterion, optimizer=None)
        print(f"Epoch {epoch:02d}: train_acc={train_acc:.4f}, val_acc={val_acc:.4f}")

        # ---- your early stopping block (with saving best) ----
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            epochs_no_improve = 0
            best_model_state = copy.deepcopy(model.state_dict())  # keep BEST weights
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= PATIENCE:
                print("Early stopping")
                break

    # restore best weights before logging result (so this combo truly reflects its best)
    if best_model_state is not None:
        model.load_state_dict(best_model_state)

    results.append({"batch_size": bs, "lr": lr, "val_acc": best_val_acc})




🔍 Testing BATCH=32, LR=0.0005
Epoch 01: train_acc=0.6003, val_acc=0.7943
Epoch 02: train_acc=0.8252, val_acc=0.8411
Epoch 03: train_acc=0.8705, val_acc=0.8839
Epoch 04: train_acc=0.9017, val_acc=0.9063
Epoch 05: train_acc=0.9237, val_acc=0.8900
Epoch 06: train_acc=0.9386, val_acc=0.9022
Early stopping

🔍 Testing BATCH=32, LR=0.001
Epoch 01: train_acc=0.6771, val_acc=0.8534
Epoch 02: train_acc=0.8490, val_acc=0.8778
Epoch 03: train_acc=0.8961, val_acc=0.8982
Epoch 04: train_acc=0.9214, val_acc=0.9124
Epoch 05: train_acc=0.9450, val_acc=0.9022
Epoch 06: train_acc=0.9656, val_acc=0.8982
Early stopping

🔍 Testing BATCH=32, LR=0.002
Epoch 01: train_acc=0.7124, val_acc=0.8228
Epoch 02: train_acc=0.8594, val_acc=0.8676
Epoch 03: train_acc=0.9128, val_acc=0.8656
Epoch 04: train_acc=0.9402, val_acc=0.8574
Early stopping

🔍 Testing BATCH=32, LR=0.003
Epoch 01: train_acc=0.7251, val_acc=0.8513
Epoch 02: train_acc=0.8449, val_acc=0.8493
Epoch 03: train_acc=0.9096, val_acc=0.8676
Epoch 04: train_a

NameError: name 'pd' is not defined

In [27]:
df_results = pd.DataFrame(results).sort_values("val_acc", ascending=False)
best = df_results.iloc[0]
print("\n🏆 Best configuration:")
print(best)
print("\nAll results (sorted):\n", df_results.reset_index(drop=True))


🏆 Best configuration:
batch_size    32.000000
lr             0.001000
val_acc        0.912424
Name: 1, dtype: float64

All results (sorted):
     batch_size      lr   val_acc
0           32  0.0010  0.912424
1          128  0.0005  0.910387
2           64  0.0005  0.908350
3           32  0.0005  0.906314
4          128  0.0020  0.902240
5           64  0.0010  0.898167
6          128  0.0010  0.892057
7           64  0.0020  0.892057
8           64  0.0030  0.885947
9           32  0.0030  0.883910
10         128  0.0030  0.881874
11          32  0.0020  0.867617


In [ ]:
# --- 1) Fix best batch size & lr from your previous grid search result `best` ---
best_batch_size = int(best["batch_size"])
best_lr = float(best["lr"])
print(f"\n✅ Using best hyperparams from LR/Batch search -> batch_size={best_batch_size}, lr={best_lr}")

# --- 2) Hidden-dim search using fixed (batch_size, lr) ---

# Search space (tweak as you like)
hidden_dims = [64, 96, 128, 192, 256, 384, 512]

results_hd = []

# Rebuild iterators ONCE with best batch size
train_iter, valid_iter = data.BucketIterator.splits(
    (train_data, valid_data),
    batch_size=best_batch_size,
    sort_key=lambda x: len(x.text),
    sort_within_batch=True,
    device=device
)

for hd in hidden_dims:
    print(f"\n🧪 Testing hidden_dim={hd}")

    # fresh embedding weights for each trial
    emb_layer = nn.Embedding.from_pretrained(
        embedding_layer.weight.data.clone(), freeze=False, padding_idx=PAD_IDX
    )

    model = RNNClassifier(
        embedding_layer=emb_layer,
        hidden_dim=hd,
        num_layers=NUM_LAYERS,
        bidirectional=BIDIRECTIONAL,
        dropout=DROPOUT,
        num_classes=NUM_CLASSES
    ).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=best_lr)

    best_val_acc_hd = -1.0
    epochs_no_improve = 0
    best_model_state_hd = None

    for epoch in range(1, MAX_EPOCHS + 1):
        train_loss, train_acc = epoch_run(train_iter, model, criterion, optimizer)
        val_loss, val_acc = epoch_run(valid_iter, model, criterion, optimizer=None)
        print(f"Epoch {epoch:02d}: train_acc={train_acc:.4f}, val_acc={val_acc:.4f}")

        # Early stopping + save-best (your pattern)
        if val_acc > best_val_acc_hd:
            best_val_acc_hd = val_acc
            epochs_no_improve = 0
            best_model_state_hd = copy.deepcopy(model.state_dict())
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= PATIENCE:
                print("Early stopping")
                break

    # Restore best for this hidden_dim before recording
    if best_model_state_hd is not None:
        model.load_state_dict(best_model_state_hd)

    results_hd.append({"hidden_dim": hd, "val_acc": best_val_acc_hd})


In [ ]:
# --- 3) Pick best hidden dim, print & save variable ---
df_hd = pd.DataFrame(results_hd).sort_values("val_acc", ascending=False).reset_index(drop=True)
best_hidden_dim = int(df_hd.loc[0, "hidden_dim"])

print("\n🏆 Best hidden_dim configuration:")
print(df_hd.loc[0])
print("\nAll hidden_dim results (sorted):")
print(df_hd)

# Variables now set for subsequent training:
print(f"\n👉 Final choice: best_batch_size={best_batch_size}, best_lr={best_lr}, best_hidden_dim={best_hidden_dim}")